In [1]:
%logstart -o -t

import sys

import numpy as np
#in case we need to repeat experiment
np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')


import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
#from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV


#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=4

#Sample Size
sampleN = 2000

import time

#Enable Optimization Algorithms
enable_grid_search = False
svm_c = 1
svm_gamma = 1
enable_rf_features = True
feature_all = False
defaultFeatures = ['P_AGE', 'V_YEAR', 'C_HOUR', 'C_YEAR', 'C_MNTH', 'C_CONF', 'C_WDAY', 'C_VEHS', 'P_USER', 'P_SEX']
enable_kmeans = True
enable_logistic_regression_l1 = True

# Enable Algorithms
enable_logistic_regression = True
enable_decision_tree = True
enable_svm = True
enable_knn = True
enable_mlp = True
enable_rf = True

#Multiclass classification, binary if falase
multiclass = True

Activating auto-logging. Current session state plus future input saved.
Filename       : ipython_log.py
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarn

In [2]:
print("Sample size: {}".format(sampleN))

if multiclass:
    print("Multi-Class Classification: Enabled")
else:
    print("Multi-Class Classification: Disabled")

if enable_grid_search:
    print("Grid Search: Enabled")
else:
    print("Grid Search: Disabled")

if enable_rf_features:
    print("Feature Selection by RandomForest: Enabled")
else:
    print("Feature Selection by RandomForest: Disabled")
    print("Default Feature: {}".format(defaultFeatures))

if feature_all:
    print("All Features: Enabled")
else:
    print("All Features: Disabled")
    
if enable_kmeans:
    print("K-means: Enabled")
else:
    print("K-means: Disabled")

if enable_logistic_regression:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_decision_tree:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")
    
if enable_rf:
    print("Random Forest: Enabeld")
else:
    print("Random Forest: Disabled")

Sample size: 2000
Multi-Class Classification: Enabled
Grid Search: Disabled
Feature Selection by RandomForest: Enabled
All Features: Disabled
K-means: Enabled
Logistic Regression: Enabled
Decision Tree: Enabled
Support Vector Machines: Enabled
KNN: Enabled
MLP: Enabled
Random Forest: Enabeld


In [3]:
#sys.stdout=open("Initial_Binary_Model2.out", "w")

In [4]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ))

Sun Nov 11 13:05:45 2018


In [5]:
#df = pd.read_csv('data01_simple.csv', engine = 'python')
df = pd.read_csv('data01_clean.csv', engine = 'python')
print(df.head(2))

   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0    1999       1       1      20       2      34       8       1       5   
1    1999       1       1      20       2      34       2       1       5   

   C_RALN   ...    V_ID  V_TYPE  V_YEAR  P_ID  P_SEX  P_AGE  P_PSN  P_SAFE  \
0       3   ...       1       6    1990     1      1     41     11       2   
1       3   ...       2       1    1987     1      1     19     11       2   

   P_USER  P_ISEV  
0       1       1  
1       1       1  

[2 rows x 21 columns]


In [6]:
print(df.isnull().sum().sum())

0


In [7]:
print(df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


In [8]:
df_cat = df.astype('category').copy()

In [9]:
df_int = df.astype('int').copy()

In [10]:
print(df.columns)

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_ID', 'V_TYPE', 'V_YEAR',
       'P_ID', 'P_SEX', 'P_AGE', 'P_PSN', 'P_SAFE', 'P_USER', 'P_ISEV'],
      dtype='object')


## Convert Class Variable to Binary if multi class disabled

In [11]:
## Convert Class Variable to Binary
### Merge Injury and Fatality as a single class
### we will compare the results.
if multiclass:
    #Undersample the majority for the 3 class evaluation
    
    df_class = df_cat.copy()
    
    # subset fatal class
    is_fatal =  df_class['P_ISEV']==3
    is_fatal_count = is_fatal.sum()
    print("Number of Fatal: {}".format(is_fatal_count))
    df_class_fatal = df_class[is_fatal]
    print(df_class_fatal.head(2))
    
    # subset injury class
    is_injury =  df_class['P_ISEV']==2
    is_injury_count = is_injury.sum()
    print("Number of Injury: {}".format(is_injury_count))
    df_class_injury = df_class[is_injury]
    print(df_class_injury.head(2))
    
    # subset non_injury class
    is_safe =  df_class['P_ISEV']==1
    is_safe_count = is_safe.sum()
    print("Number of Non-Injury: {}".format(is_safe_count))
    df_class_safe = df_class[is_safe]
    print(df_class_safe.head(2))
    
    # get the size of fatal datafram
    min_size = df_class_fatal.index.size
    print("Size of Fatal Subset: {}".format(min_size))
    
    # get size of injury
    print("Size of injury Subset: {}".format(df_class_injury.index.size))
    
    # size of non-fatal
    print("Size of non-fatal Subset: {}".format(df_class_safe.index.size))
    
    # randomly sample n number of injury and no injury and append to fatal
    df_class_injury_select = df_class_injury.sample(n=min_size)
    print("Shape of injury sampled dataframe: {}".format(df_class_injury_select.shape))
    df_class_safe_select = df_class_safe.sample(n=min_size)
    print("Shape of nom-injury sampled dataframe: {}".format(df_class_safe_select.shape))
    
    #concat the three dataframes
    df_sample = pd.concat([df_class_fatal, df_class_injury_select, df_class_safe_select])
    print(df_sample.shape)
    
    #perform the conversion in two steps to avoid any unwanted side effects
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'2'})
    print((df_sample['P_ISEV']=='0').sum())
    print((df_sample['P_ISEV']=='1').sum())
    print((df_sample['P_ISEV']=='2').sum())
    print(df_sample['P_ISEV'].unique())
else:
    df_class = df_cat.copy()

    #perform the conversion in two steps to avoid any unwanted side effects
    df_class['P_ISEV'] = df_class['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_class['P_ISEV'] = df_class['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'1'})
    print((df_class['P_ISEV']=='0').sum())
    print((df_class['P_ISEV']=='1').sum())
    print(df_class['P_ISEV'].unique())
    
    df_sample = df_class.sample(n=sampleN)

Number of Fatal: 43559
    C_YEAR C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN  \
47    1999      1      1     23      1      4      2      1      1      4   
151   1999      1      1     12      1      1      2      1      1      1   

     ...   V_ID V_TYPE V_YEAR P_ID P_SEX P_AGE P_PSN P_SAFE P_USER P_ISEV  
47   ...      1      6   1997    1     1    44    11      2      1      3  
151  ...     99      1   2017    1     1    15    99      2      3      3  

[2 rows x 21 columns]
Number of Injury: 3351616
  C_YEAR C_MNTH C_WDAY C_HOUR C_VEHS C_CONF C_RCFG C_WTHR C_RSUR C_RALN  \
2   1999      1      1     20      2     34      2      1      5      3   
4   1999      1      1      8      1      1      2      5      3      6   

   ...   V_ID V_TYPE V_YEAR P_ID P_SEX P_AGE P_PSN P_SAFE P_USER P_ISEV  
2  ...      2      1   1987    2     0    20    13      2      2      2  
4  ...     99      1   2017    1     1     5    99      2      3      2  

[2 rows x 21 column

## Split Training and Testing for Binary class

In [12]:
#Split between data and class
Y = df_sample[df_sample.columns[-1]]
X = df_sample[df_sample.columns[0:df_sample.columns.size -1]]

#### Split Test(70%) and Train (30%) for Bianry class 

In [13]:
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [14]:
#print(Xbinary_train, Xbinary_test, Ybinary_train, Ybinary_test)

## Write cleaned data to file for future use.

In [15]:
#lets write the datafile for future use
df_sample.to_csv('cleansimple_binary.csv', encoding='utf-8', index=False)

## Clustering based on K-Means Clustering

In [16]:
if enable_kmeans:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    print("K-Means Clustering: Start")
    kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=100, tol=1e-04, verbose= verbose_level)
    print("K-Means Clustering: Build")
    ykm = kmeans.fit(X)
    
    if pyscript:
        print(ykm.cluster_centers_)
        print(ykm.labels_)
    else:
        display(ykm.cluster_centers_)
        display(ykm.labels_)
    
    print("K-Means Clustering: End")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))


Sun Nov 11 13:09:20 2018
K-Means Clustering: Start
K-Means Clustering: Build
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 211280608.70485866
start iteration
done sorting
end inner loop
Iteration 1, inertia 151581533.6000452
start iteration
done sorting
end inner loop
Iteration 2, inertia 86971887.87358797
start iteration
done sorting
end inner loop
Iteration 3, inertia 85350756.78652638
start iteration
done sorting
end inner loop
Iteration 4, inertia 83135763.23882091
start iteration
done sorting
end inner loop
Iteration 5, inertia 82887806.17937374
start iteration
done sorting
end inner loop
Iteration 6, inertia 82853800.11943491
start iteration
done sorting
end inner loop
Iteration 7, inertia 82842080.79316713
start iteration
done sorting
end inner loop
Iteration 8, inertia 82838456.41562855
start iteration
done sorting
end inner loop
Iteration 9, inertia 82837204.56623851
start iteration
done sorting
end inner loop
Iteration 10, inertia 82

## Feature selection using Random Forest

### Feature Selection 

### Get the inportant features from random forest

In [17]:
if enable_rf_features:
    print("Random Forest Feature Selection: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1, verbose=verbose_level)
    print("Random Forest Feature Selection: Fit Start")
    forest.fit(X, Y)
    print("Random Forest Feature Selection: Fit")

    importFeatures = forest.feature_importances_
    print("Random Forest Feature Selection: Feature Importance")
    print(importFeatures)
    
    indices = np.argsort(importFeatures)[::-1]
    print(indices)
    featureLabel = X.columns[0:]
    print(featureLabel)
    rankedFeature = []
    for f in range(X.shape[1]):
        rankedFeature.append(featureLabel[indices[f]])
        print("%2d) %-*s %f" % (f+1, 30,  featureLabel[indices[f]], importFeatures[indices[f]]))
    print(rankedFeature)

Random Forest Feature Selection: Start
Sun Nov 11 13:09:24 2018
Random Forest Feature Selection: Fit Start
building tree 1 of 250building tree 2 of 250

building tree 3 of 250
building tree 4 of 250
building tree 5 of 250
building tree 6 of 250
building tree 7 of 250building tree 8 of 250

building tree 9 of 250
building tree 10 of 250
building tree 11 of 250
building tree 12 of 250
building tree 13 of 250
building tree 14 of 250
building tree 15 of 250
building tree 16 of 250
building tree 17 of 250
building tree 18 of 250
building tree 19 of 250
building tree 20 of 250
building tree 21 of 250
building tree 22 of 250building tree 23 of 250

building tree 24 of 250


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.6s


building tree 25 of 250building tree 26 of 250

building tree 27 of 250
building tree 28 of 250
building tree 29 of 250
building tree 30 of 250
building tree 31 of 250
building tree 32 of 250
building tree 33 of 250
building tree 34 of 250
building tree 35 of 250
building tree 36 of 250
building tree 37 of 250
building tree 38 of 250
building tree 39 of 250
building tree 40 of 250
building tree 41 of 250
building tree 42 of 250
building tree 43 of 250
building tree 44 of 250
building tree 45 of 250
building tree 46 of 250
building tree 47 of 250
building tree 48 of 250
building tree 49 of 250
building tree 50 of 250
building tree 51 of 250
building tree 52 of 250
building tree 53 of 250
building tree 54 of 250
building tree 55 of 250
building tree 56 of 250
building tree 57 of 250
building tree 58 of 250
building tree 59 of 250building tree 60 of 250

building tree 61 of 250
building tree 62 of 250
building tree 63 of 250
building tree 64 of 250
building tree 65 of 250
building tree 66

[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    3.6s


building tree 97 of 250
building tree 98 of 250
building tree 99 of 250
building tree 100 of 250
building tree 101 of 250
building tree 102 of 250
building tree 103 of 250
building tree 104 of 250
building tree 105 of 250
building tree 106 of 250
building tree 107 of 250
building tree 108 of 250
building tree 109 of 250
building tree 110 of 250building tree 111 of 250

building tree 112 of 250
building tree 113 of 250
building tree 114 of 250
building tree 115 of 250
building tree 116 of 250
building tree 117 of 250
building tree 118 of 250
building tree 119 of 250
building tree 120 of 250
building tree 121 of 250
building tree 122 of 250
building tree 123 of 250
building tree 124 of 250
building tree 125 of 250
building tree 126 of 250
building tree 127 of 250
building tree 128 of 250
building tree 129 of 250
building tree 130 of 250
building tree 131 of 250
building tree 132 of 250
building tree 133 of 250
building tree 134 of 250
building tree 135 of 250
building tree 136 of 250
bui

[Parallel(n_jobs=-1)]: Done 205 tasks      | elapsed:    8.6s


building tree 218 of 250
building tree 219 of 250
building tree 220 of 250
building tree 221 of 250
building tree 222 of 250
building tree 223 of 250
building tree 224 of 250
building tree 225 of 250
building tree 226 of 250
building tree 227 of 250
building tree 228 of 250
building tree 229 of 250
building tree 230 of 250
building tree 231 of 250
building tree 232 of 250
building tree 233 of 250
building tree 234 of 250
building tree 235 of 250
building tree 236 of 250
building tree 237 of 250
building tree 238 of 250
building tree 239 of 250
building tree 240 of 250
building tree 241 of 250
building tree 242 of 250
building tree 243 of 250
building tree 244 of 250
building tree 245 of 250
building tree 246 of 250
building tree 247 of 250
building tree 248 of 250
building tree 249 of 250
building tree 250 of 250
Random Forest Feature Selection: Fit
Random Forest Feature Selection: Feature Importance
[0.0834577  0.07503953 0.06160617 0.09303    0.03450744 0.09665788
 0.03058348 0.02836

[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   10.4s finished


### Reduce the number of features

In [18]:
#select features that contribute more than 0.05
if enable_rf_features:
    X_Selected = X[rankedFeature[0:10]]
else:
    if feature_all:
        X_Selected = X
    else:
        X_Selected = X[defaultFeatures]

if pyscript:
    print(X_Selected)
else:
    display(X_Selected)

t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

print("Random Forest Feature Selection: End")


        P_AGE C_CONF C_HOUR V_YEAR C_YEAR C_MNTH C_WDAY P_SAFE C_VEHS C_TRAF
47         44      4     23   1997   1999      1      1      2      1     18
151        15      1     12   2017   1999      1      1      2      1      6
258        26      1     15   2017   1999      1      1      2      1      1
307        74      2     22   1991   1999      1      1      2      1     18
503        59     31     13   1989   1999      1      1      2      2     18
513         5      1      7   2017   1999      1      1      2      1     18
524        60     35     20   1990   1999      1      1     13      2      1
527        52     21     20   1986   1999      1      1      2      2     18
640        30     31     11   1988   1999      1      1      2      2     18
641        27     31     11   1988   1999      1      1      2      2     18
642         7     31     11   1988   1999      1      1      2      2     18
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...

In [19]:
print("Split Test and Train based on Selected Features")
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X_Selected, Y, test_size=0.3, random_state=0)

Split Test and Train based on Selected Features


### SVM GridSearch for Optimal Parms

In [20]:
#This operation is computationaly expensive.
if enable_grid_search:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level)
    grid.fit(X_train, Y_train)
    print(grid.best_params_)
    svm_c = grid.best_params_.get('C')
    svm_gamma = grid.best_params_.get('gamma')
    print(grid.best_estimator_)
    grid_predictions = grid.predict(X_test)
    cfn_matrix_grid = confusion_matrix(Y_test, grid_predictions)
    print(cfn_matrix_grid)
    print(classification_report(Y_test,grid_predictions))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

## Logistic Regression Model

In [21]:
if enable_logistic_regression:
    print("Logistic Regression: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression: Fit")
    lr.fit(X_train, Y_train)
    print("Logistic Regression: Predict")
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(lr.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(lr.coef_)

    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg)
    
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression: End")

Logistic Regression: Start
Sun Nov 11 13:09:35 2018
Logistic Regression: Fit
[LibLinear]Logistic Regression: Predict
Accuracy of logistic regression classifier on train set: 0.47
Accuracy of logistic regression classifier on test set: 0.46
Logistic Regression: Intercept
[-5.09996102e-04 -5.63764791e-05  8.50339240e-04]
Logistic Regression: Coefficients
[[-0.01087864  0.00964506  0.01994049 -0.01267847  0.01231995 -0.0047745
  -0.02485683 -0.09374094  0.37538169 -0.04319582]
 [-0.00656579 -0.00109746  0.00515405  0.00315407 -0.00314781 -0.00959903
  -0.03530438  0.00750739 -0.02344563 -0.02145296]
 [ 0.01849188 -0.00304781 -0.02552123  0.00788566 -0.00871877  0.01604748
   0.06007965  0.0612583  -0.44122949  0.07717584]]
Logistic Regression: Confusion Matrix
[[8032 1150 3981]
 [6126 1329 5663]
 [3044 1059 8820]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.47      0.61      0.53     13163
          1       0.38     

### Logistic Regression with L1 Regularization

In [22]:
if enable_logistic_regression_l1:
    # with L1 regularization
    print("Logistic Regression with L1 Regularization: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(penalty='l1', C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression with L1 Regularization: Fit")
    lr.fit(X_train, Y_train)
    
    print("Logistic Regression with L1 Regularization: Predict")
    y_pred = lr.predict(X_test)
    
    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    print("Logistic Regression with L1 Regularization: Confusion Matrix")
    cnf_matrix_lg_l1 = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg_l1)
    
    print(classification_report(Y_test,y_pred))
    print("Logistic Regression with L1 Regularization: Classification Report")

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression with L1 Regularization: End")

Logistic Regression with L1 Regularization: Start
Sun Nov 11 13:09:38 2018
Logistic Regression with L1 Regularization: Fit
[LibLinear]

C:\Users\pradeep\Anaconda3\lib\site-packages\sklearn\svm\base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Logistic Regression with L1 Regularization: Predict
Accuracy of logistic regression classifier on train set: 0.47
Accuracy of logistic regression classifier on test set: 0.46
Logistic Regression with L1 Regularization: Confusion Matrix
[[8116 1062 3985]
 [6231 1229 5658]
 [3047 1031 8845]]
             precision    recall  f1-score   support

          0       0.47      0.62      0.53     13163
          1       0.37      0.09      0.15     13118
          2       0.48      0.68      0.56     12923

avg / total       0.44      0.46      0.41     39204

Logistic Regression with L1 Regularization: Classification Report
Sun Nov 11 13:13:27 2018
Logistic Regression with L1 Regularization: End


### Decision Tree

In [23]:
if enable_decision_tree:
    print("Decision Tree: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=50, random_state=0)
    print("Decision Tree: Fit")
    tree.fit(X_train, Y_train)
    
    print("Decision Tree: Predict")
    y_pred = tree.predict(X_test)
    print('Accuracy of Decision Tree classifier on train set: {:.2f}'.format(tree.score(X_train, Y_train)))
    print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))
    
    cnf_matrix_dt = confusion_matrix(Y_test, y_pred)
    print("Decision Tree: Confusion Matrix")
    print(cnf_matrix_dt)
    print("Decision Tree: Classification Report")
    print(classification_report(Y_test,y_pred))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    print("Decision Tree: End")

Decision Tree: Start
Sun Nov 11 13:13:27 2018
Decision Tree: Fit
Decision Tree: Predict
Accuracy of Decision Tree classifier on train set: 1.00
Accuracy of Decision Tree classifier on test set: 0.50
Decision Tree: Confusion Matrix
[[6421 4849 1893]
 [4805 5168 3145]
 [1844 3110 7969]]
Decision Tree: Classification Report
             precision    recall  f1-score   support

          0       0.49      0.49      0.49     13163
          1       0.39      0.39      0.39     13118
          2       0.61      0.62      0.61     12923

avg / total       0.50      0.50      0.50     39204

Sun Nov 11 13:13:28 2018
Decision Tree: End


### K-N-N

In [24]:
if enable_knn:
    print("KNN: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
    print("KNN: Fit")
    knn.fit(X_train, Y_train)
    
    print("KNN: Predict")
    y_pred = knn.predict(X_test)
    print('Accuracy of KNN classifier on train set: {:.2f}'.format(knn.score(X_train, Y_train)))
    print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_test, Y_test)))
    
    print("KNN: Confusion Matrix")
    cnf_matrix_knn = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_knn)

    print("KNN: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

    print("KNN: End")

KNN: Start
Sun Nov 11 13:13:28 2018
KNN: Fit
KNN: Predict
Accuracy of KNN classifier on train set: 0.66
Accuracy of KNN classifier on test set: 0.50
KNN: Confusion Matrix
[[7536 4110 1517]
 [5709 4565 2844]
 [2445 2871 7607]]
KNN: Classification Report
             precision    recall  f1-score   support

          0       0.48      0.57      0.52     13163
          1       0.40      0.35      0.37     13118
          2       0.64      0.59      0.61     12923

avg / total       0.50      0.50      0.50     39204

Sun Nov 11 13:13:48 2018
KNN: End


### SVM

In [ ]:
if enable_svm:
    print("SVM: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    #svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
    #svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
    svm = SVC(C=svm_c, gamma=svm_gamma, verbose = verbose_level)
    print("SVM: Fit")
    svm.fit(X_train, Y_train)
    
    print("SVM: Predict")
    y_pred = svm.predict(X_test)
    
    print('Accuracy of SVM classifier on train set: {:.2f}'.format(svm.score(X_train, Y_train)))
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))
    
    print("SVM: Confusion Matrix")
    cnf_matrix_svm = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_svm)
    
    print("SVM: Classfication Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("SVM: End")

SVM: Start
Sun Nov 11 13:13:48 2018
SVM: Fit
[LibSVM]

## ANN - Multilayer Perceptron

In [ ]:
if enable_mlp:
    print("Multilayer Preceptron: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    #mlpc = MLPClassifier(alpha=1)
    mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=100, verbose=verbose_level)
    
    #mlp = multilayer_perceptron(n_hidden =2, activation='logistic', algorithm='sgd', random_state=3)
    print("Multilayer Preceptron: fit")
    mlpc.fit(X_train, Y_train)
    
    print("Multilayer Preceptron: Predict")
    y_pred = mlpc.predict(X_test)

    print('Accuracy of Multilayer Perceptron classifier on train set: {:.2f}'.format(mlpc.score(X_train, Y_train)))
    print('Accuracy of Multilayer Perceptron classifier on test set: {:.2f}'.format(mlpc.score(X_test, Y_test)))
    
    print("Multilayer Preceptron: Confusion Matrix")
    cnf_matrix_mlp = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_mlp)
    
    print("Multilayer Preceptron: Classificiation Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Multilayer Preceptron: End")

### Random forest

In [ ]:
if enable_rf:
    print("Ensemble (Bagging): Random Forest: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=0, n_jobs=2, verbose=verbose_level)
    print("Ensemble (Bagging): Random Forest: Fit")
    forest.fit(X_train, Y_train)
    
    print("Ensemble (Bagging): Random Forest: Predict")
    y_pred = forest.predict(X_test)
    
    print('Accuracy of RandomForest classifier on train set: {:.2f}'.format(forest.score(X_train, Y_train)))
    print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))
    
    print("Ensemble (Bagging): Random Forest: Confusion Matrix")
    cnf_matrix_rf = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_rf)
    
    print("Ensemble (Bagging): Random Forest: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    print("Ensemble (Bagging): Random Forest: End")

In [ ]:

#check sigmoid and rbf
#from sklearn.ensemble import BaggingClassifier
#from sklearn.svm import SVC
#clf = BaggingClassifier(SVC(C=1.0,
#        cache_size=200,
#        class_weight=None,
#        coef0=0.0,
#        decision_function_shape=None,
#        degree=3,
#        gamma='auto',
#        kernel='linear',
#        max_iter=-1,
#        probability=False,
#        random_state=None,
#        shrinking=True,
#        tol=0.001,
#        verbose=False,
#        ))

In [ ]:
#clf.fit(X_train, Y_train)

In [ ]:
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

In [ ]:
#sys.stdout.close()
%logstop